In [2]:
%pip install pandas

  Using cached pandas-2.3.3-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)
  Using cached numpy-2.3.4-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.3.3-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl (12.4 MB)
Using cached numpy-2.3.4-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.6 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]
Note: you may need to restart the kernel to use updated packages.


In [13]:
import pandas as pd
import json

# 1. Create a clean JSON framework from the sample
with open('dsld_sample_data.json', 'r') as f:
    sample_data = json.load(f)

# "Wash" the sample data to create a clean framework
json_framework = sample_data.copy()

# 2. Extract data from CSV
csv_file = 'supplement_build.csv'
df = pd.read_csv(csv_file)

# Get blend name and servings from the first row of the CSV
blend_name = df['Blend Name'].iloc[0] if not df.empty else ""
servings = df['Servings Per Container'].iloc[0] if not df.empty else ""

# 3. Pre-populate data
json_framework.update({
    "src": "",
    "id": None,
    "nhanesId": "",
    "bundleName": "",
    "fullName": blend_name,
    "brandName": "Supp Tools",
    "brandIpSymbol": "",
    "upcSku": "850051590427",
    "productVersionCode": "10001",
    "pdf": "",
    "thumbnail": "",
    "servingsPerContainer": str(servings),
    "percentDvFootnote": "Percent Daily Value (%DV) are based on a 2,000 calorie diet.",
    "hasOuterCarton": None,
    "labelRelationships": [],
    "contacts": [
        {
            "contactId": None,
            "text": "Distributed by",
            "types": ["Distributor"],
            "contactDetails": {
                "src": "",
                "id": None,
                "name": "Supp Tools",
                "streetAddress": "",
                "city": "Chicago",
                "state": "IL",
                "country": "US",
                "zipCode": "60606",
                "phoneNumber": "1.800.123.4567",
                "email": "",
                "webAddress": "www.supptools.com"
            }
        }
    ],
    "netContents": [],
    "statements": [
        {
            "type": "Suggested/Recommended/Usage/Directions",
            "notes": "Suggested use:\\r\\nTake one serving (approx. 1 scoop), mixed with 8 fl. oz. of water 20-30 minutes before training."
        },
        {
            "type": "Precautions re: Children",
            "notes": "Warning:\\r\\nThis product is only intended to be consumed by healthy adults, 18 years of age or older."
        },
        {
            "type": "Precautions re: Pregnant or Nursing or Prescription Medications",
            "notes": "Do not use this product if you are pregnant or nursing. Do not take this product before first consulting a licensed, qualified, healthcare professional, especially if you are taking any other dietary supplement, prescription drug or over-the-counter medication; or, if you suspect you have or have been treated for, diagnosed with, or have a family history of, any medical condition."
        },
        {
            "type": "Precautions re: All Other",
            "notes": "Discontinue use 2 weeks prior to surgery. Immediately discontinue use and contact a medical doctor if you experience any adverse reaction to this product. Do not use if safety seal is broken or missing."
        },
        {
            "type": "Storage",
            "notes": "Store in a cool, dry place."
        },
        {
            "type": "Precautions re: Children",
            "notes": "Keep out of reach of children."
        },
        {
            "type": "General Statements: All Other Content",
            "notes": "This product is sold by weight, not volume. The density of the powder varies, and significant settling of powder may occur during shipping and handling."
        },
        {
            "type": "General Statements: All Other Content",
            "notes": "This product contains the servings indicated when measured exactly by weight."
        },
        {
            "type": "Formulation re: Other",
            "notes": "Made in the USA using strategically sourced domestic and imported ingredients and components."
        },
        {
            "type": "FDA Disclaimer Statement",
            "notes": "These statements have not been evaluated by the Food and Drug administration. This product is not intended to diagnose, treat, cure, or prevent any disease."
        },
        {
            "type": "General Statements: All Other Content",
            "notes": "iD\\r\\nSeries"
        },
        {
            "type": "Formulation re: Other",
            "notes": "Explosive pre-workout and cutting formula"
        },
        {
            "type": "FDA Statement of Identity",
            "notes": "Dietary Supplement"
        }
    ],
    "claims": [],
    "events": [],
    "ingredientRows": [],
    "otheringredients": {"text": None, "ingredients": []},
    "offMarket": 0,
    "entryDate": "",
    "physicalState": {
        "langualCode": "E0162",
        "langualCodeDescription": "Powder"
    },
    "servingSizes": [
        {
            "order": 1,
            "minQuantity": 1,
            "maxQuantity": 1,
            "unit": "Gram(s)",
            "notes": "Approx. 1 Scoop",
            "inSFB": True
        }
    ],
    "targetGroups": [
        "Adult (18 - 50 Years)",
        "Women (not pregnant or lactating)"
    ],
    "productType": {
        "langualCode": "A1325",
        "langualCodeDescription": "Other Combinations"
    }
})

# We will keep servingSizes, userGroups, and physicalState as they might be useful defaults
# If these need to be dynamic, they can be cleared here as well.

# 4. Populate ingredients list from CSV
ingredients_list = []
# Use the first serving size as a default
serving_size = json_framework.get('servingSizes', [{}])[0]
serving_size_quantity = serving_size.get('minQuantity')
serving_size_unit = serving_size.get('unit')

for index, row in df.iterrows():
    ingredient_data = {
        "order": index + 1,
        "ingredientId": None,
        "description": None,
        "notes": "",
        "quantity": [
            {
                "servingSizeOrder": 1,
                "servingSizeQuantity": serving_size_quantity,
                "operator": "=",
                "quantity": row['Quantity'],
                "unit": row['Unit'],
                "dailyValueTargetGroup": [],
                "servingSizeUnit": serving_size_unit
            }
        ],
        "nestedRows": [],
        "name": row['Ingredient Name'],
        "category": None,
        "ingredientGroup": None,
        "uniiCode": None,
        "alternateNames": [],
        "forms": []
    }
    ingredients_list.append(ingredient_data)

# 5. Inject the list of ingredients into the JSON framework
json_framework['ingredientRows'] = ingredients_list

# 6. Save the new JSON data
output_file = 'supplement_build_refactored.json'
with open(output_file, 'w') as f:
    json.dump(json_framework, f, indent=2)

print(f"Successfully created {output_file}")
print("---")
print("Review of populated data:")
print(f"Full Name: {json_framework['fullName']}")
print(f"Brand Name: {json_framework['brandName']}")
print(f"Servings: {json_framework['servingsPerContainer']}")
print(f"UPC: {json_framework['upcSku']}")
print(f"Version Code: {json_framework['productVersionCode']}")
print(f"DV Footnote: {json_framework['percentDvFootnote']}")

Successfully created supplement_build_refactored.json
---
Review of populated data:
Full Name: Sample
Brand Name: Supp Tools
Servings: 30
UPC: 850051590427
Version Code: 10001
DV Footnote: Percent Daily Value (%DV) are based on a 2,000 calorie diet.


In [18]:
# "Wash" the sample data to create a clean framework
json_framework = sample_data.copy()

# 3. Pre-populate data
json_framework.update({
    "src": "",
    "id": None,
    "nhanesId": "",
    "bundleName": "",
    "fullName": "",
    "brandName": "Supp Tools",
    "brandIpSymbol": "",
    "upcSku": "850051590427",
    "productVersionCode": "10001",
    "pdf": "",
    "thumbnail": "",
    "servingsPerContainer": "",
    "percentDvFootnote": "Percent Daily Value (%DV) are based on a 2,000 calorie diet.",
    "hasOuterCarton": None,
    "labelRelationships": [],
    "contacts": [
        {
            "contactId": None,
            "text": "Distributed by",
            "types": ["Distributor"],
            "contactDetails": {
                "src": "",
                "id": None,
                "name": "Supp Tools",
                "streetAddress": "",
                "city": "Chicago",
                "state": "IL",
                "country": "US",
                "zipCode": "60606",
                "phoneNumber": "1.800.123.4567",
                "email": "",
                "webAddress": "www.supptools.com"
            }
        }
    ],
    "netContents": [],
    "statements": [
        {
            "type": "Suggested/Recommended/Usage/Directions",
            "notes": "Suggested use:\\r\\nTake one serving (approx. 1 scoop), mixed with 8 fl. oz. of water 20-30 minutes before training."
        },
        {
            "type": "Precautions re: Children",
            "notes": "Warning:\\r\\nThis product is only intended to be consumed by healthy adults, 18 years of age or older."
        },
        {
            "type": "Precautions re: Pregnant or Nursing or Prescription Medications",
            "notes": "Do not use this product if you are pregnant or nursing. Do not take this product before first consulting a licensed, qualified, healthcare professional, especially if you are taking any other dietary supplement, prescription drug or over-the-counter medication; or, if you suspect you have or have been treated for, diagnosed with, or have a family history of, any medical condition."
        },
        {
            "type": "Precautions re: All Other",
            "notes": "Discontinue use 2 weeks prior to surgery. Immediately discontinue use and contact a medical doctor if you experience any adverse reaction to this product. Do not use if safety seal is broken or missing."
        },
        {
            "type": "Storage",
            "notes": "Store in a cool, dry place."
        },
        {
            "type": "Precautions re: Children",
            "notes": "Keep out of reach of children."
        },
        {
            "type": "General Statements: All Other Content",
            "notes": "This product is sold by weight, not volume. The density of the powder varies, and significant settling of powder may occur during shipping and handling."
        },
        {
            "type": "General Statements: All Other Content",
            "notes": "This product contains the servings indicated when measured exactly by weight."
        },
        {
            "type": "Formulation re: Other",
            "notes": "Made in the USA using strategically sourced domestic and imported ingredients and components."
        },
        {
            "type": "FDA Disclaimer Statement",
            "notes": "These statements have not been evaluated by the Food and Drug administration. This product is not intended to diagnose, treat, cure, or prevent any disease."
        },
        {
            "type": "General Statements: All Other Content",
            "notes": "iD\\r\\nSeries"
        },
        {
            "type": "Formulation re: Other",
            "notes": "Explosive pre-workout and cutting formula"
        },
        {
            "type": "FDA Statement of Identity",
            "notes": "Dietary Supplement"
        }
    ],
    "claims": [],
    "events": [],
    "ingredientRows": [],
    "otheringredients": {"text": None, "ingredients": []},
    "offMarket": 0,
    "entryDate": "",
    "physicalState": {
        "langualCode": "E0162",
        "langualCodeDescription": "Powder"
    },
    "servingSizes": [
        {
            "order": 1,
            "minQuantity": 1,
            "maxQuantity": 1,
            "unit": "Gram(s)",
            "notes": "Approx. 1 Scoop",
            "inSFB": True
        }
    ],
    "targetGroups": [
        "Adult (18 - 50 Years)",
        "Women (not pregnant or lactating)"
    ],
    "productType": {
        "langualCode": "A1325",
        "langualCodeDescription": "Other Combinations"
    }
})

output_file = 'dsld_json_framework.json'
with open(output_file, 'w') as f:
    json.dump(json_framework, f, indent=2)